# Potrebna muzicka teorija

In [ ]:
# Keys
#              0    1     2    3     4    5     6    7    8     9    10    11
piano_keys = ['c', 'cs', 'd', 'ds', 'e', 'f', 'fs', 'g', 'gs', 'a', 'as', 'b']

# Modes
modes = {
    "major" : [0, 2, 4, 5, 7, 9, 11],
    "minor" : [0, 2, 3, 5, 7, 8, 10]
}

In [ ]:
# Trajanje note u sekundama
def note_duration(tempo, note_length):
    bar_duration = 60 / tempo * 4
    return note_length * bar_duration

In [ ]:
# Note u skali
def scale(root, mode):
    scl = []
    n = piano_keys.index(root)

    for i in modes[mode]:
        scl.append(piano_keys[(i+n)%12])

    return scl

# Sviranje dobijene melodije

In [ ]:
import pygame as pg
import time
from threading import Thread

In [ ]:
pg.mixer.init()
pg.init()

In [ ]:
def path(note):
    # return 'Notes/' + note[-1] + '-' + note[:-1] + '.wav'
    return 'Piano/' + note + '.wav'

In [ ]:
def play_note(note, duration, volume):
    pg.mixer.music.set_volume(volume)
    pg.mixer.music.load(path(note))
    pg.mixer.music.play()
    time.sleep(note_duration(150, duration))

# Genetski algoritam

In [ ]:
import random

In [ ]:
class Melody:

    def __init__(self, num_bars, scale_root, scale_mode):
        self.num_bars = num_bars
        self.scale_root = scale_root
        self.scale_mode = scale_mode
        self.notes_in_scale = []
        self.notes = self.generate_melody()
        self.fitness = self.calc_fitness()

    def generate_melody(self):
        n = 0
        notes = []
        self.notes_in_scale = scale(self.scale_root, self.scale_mode)
        while n < self.num_bars:
            #octave = random.randint(3,6)
            octave = 5
            note = random.choice(self.notes_in_scale)
            length = min(self.num_bars - n, random.choice([1, 1/2, 1/4, 1/8, 3/4, 3/8]))
            n += length
            volume = random.random()
            notes.append([note+str(octave), length, volume])

        return notes

    def play(self):
        pg.mixer.set_num_channels(len(self.notes))
        th = {}
        n = 0
        for i in self.notes:
            th[n] = Thread(target=play_note, args=(i[0], i[1], 1))
            th[n].start()
            th[n].join()
            n += 1
            

    def calc_fitness(self):
        self.play()
        rating = int(input("Rating [0-5]: "))
        return rating

In [ ]:
def selection(population, tournament_size):
    chosen = random.sample(population, tournament_size)
    return max(chosen, key=lambda x: x.fitness)

In [ ]:
def crossover(parent1, parent2, child1, child2):
    
    random_pos = random.randrange(0, len(parent1.notes))
    
    child1.notes[:random_pos] = parent1.notes[:random_pos]
    child1.notes[random_pos:] = parent2.notes[random_pos:]
    
    child2.notes[:random_pos] = parent2.notes[:random_pos]
    child2.notes[random_pos:] = parent1.notes[random_pos:]

In [ ]:
def mutation(melody, mutation_prob):
    for i in range(len(melody.notes)):
        prob = random.random()
        if prob < mutation_prob:
            new_note = random.choice(melody.notes_in_scale)
            #octave = random.randint(3,6)
            melody.notes[i][0] = new_note+str(5)

In [ ]:
def genetic_algorithm(num_bars=8, scale_root="c", scale_mode="major", population_size=4, num_generations=4, 
                      elitism_size=2, tournament_size=2, mutation_probability=0.5):

    population = [Melody(num_bars, scale_root, scale_mode) for _ in range(population_size)]
    new_population = population.copy()

    for i in range(num_generations):
        
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        
        for j in range(elitism_size, population_size, 2):
            
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size)
            
            crossover(parent1, parent2, child1=new_population[j], child2=new_population[j+1])
    
            mutation(new_population[j], mutation_probability)
            mutation(new_population[j+1], mutation_probability)
            
            new_population[j].fitness = new_population[j].calc_fitness()
            new_population[j+1].fitness = new_population[j+1].calc_fitness()
    
        population = new_population.copy()

    return max(population, key=lambda x: x.fitness)

# Main

In [ ]:
scale_root = input("Scale root [ex. c (for C), cs (for C#)]: ")
scale_mode = input("Scale mode [major or minor]: ")
num_bars = int(input("Number of bars: "))

melody = genetic_algorithm(num_bars=num_bars, scale_root=scale_root, scale_mode=scale_mode)

In [ ]:
melody.play()